In [1]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import
from __future__ import unicode_literals

import os
import sys

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl

%matplotlib notebook

In [2]:
try:
    import seaborn as sns
    sns.set()
    sns.set_style("whitegrid")
    sns.set_context("poster")
except ImportError as e:
    print("Cannot import seaborn.")
    print("Consider installing it for nice plot !")

mpl.rcParams['figure.figsize'] = [12.0, 9.0]
mpl.rcParams['figure.dpi'] = 80
mpl.rcParams['savefig.dpi'] = 100

mpl.rcParams['font.size'] = 22
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['ytick.labelsize'] = 18
mpl.rcParams['xtick.labelsize'] = 18
mpl.rcParams['legend.fontsize'] = 'large'
mpl.rcParams['figure.titlesize'] = 'medium'


# Introduction


## Hough Transform

Consider a track pattern recognition method using the Hough Tramsform in polar system. In this system a circular track can be parametrized as follow:

$$
r = 2r_{0}Cos(\phi - \theta)
$$

where:
* $r$ and $\phi$ : are coordinates of a hit in the polar system.
* $r_{0}$ and $\theta$ : are coordinates of a center of a circular track in the polar system.

A linear track corresponds to the $r_{0} = \infty$.

Transformation of cartesian coordinates of a hit to polar coordinates defined as:

$$
\phi = arctan(\frac{y}{x})
$$
$$
r = \sqrt{x^{2} + y^{2}}
$$


The Hough Transform converts a hit in $(r, \phi)$ space to a curve in $(\frac{1}{r_{0}}, \theta)$ space of the track parameters as follow:

$$
\frac{1}{r_{0}} = \frac{2Cos(\phi - \theta)}{r}
$$

A linear track in this space represents as $(0, \theta)$ point.



however, there are 3 dimensions: x, y, z. Thus, the track pattern recognition will be performed in cylindrical coordinate systems: $\phi$, r, z. For the simplicity (but you can create your own parameter) we suppose that for 3D tracks:

$$
\gamma=\frac{z}{r}=const
$$

which is true for high-PT tracks.

This section demonstrates the track pattern recognition method using Hough Transfrom described above and histogramming technique. In this technique each 'hot' bin represents one recognized track as it is shown in the figure:

<img src="pic/hough.png" /> <br>

To assign only one track lable to a hit, only bins with the highest number of hits are selected. But there is one additional requirement for the bins: these bins must not share hits. Please, look the method script for details.

# Clustering 

In [3]:
def cartesian_to_cylindrical(x, y, z):
    
    r = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    z = z
    
    return r, phi, z



def create_hough_matrix(hit_id, x, y, z, r, phi, N):

    hits_dict = {'HitID': [], 'X': [], 'Y': [], 'Z': [], 'R': [], 'Phi': [], 'Theta': []}
    theta = list(np.linspace(-1*np.pi, 1*np.pi, N))
    
    for i in range(len(hit_id)):
        
        hits_dict['HitID'] += [hit_id[i]] * len(theta)
        hits_dict['X'] += [x[i]] * len(theta)
        hits_dict['Y'] += [y[i]] * len(theta)
        hits_dict['Z'] += [z[i]] * len(theta)
        hits_dict['R'] += [r[i]] * len(theta)
        hits_dict['Phi'] += [phi[i]] * len(theta)
        hits_dict['Theta'] += theta
        
    hough_matrix = pd.DataFrame(hits_dict)

    
    return hough_matrix




def add_r0_inv(hough_matrix):
    
    hough_matrix['R0Inv'] = (2. * np.cos(hough_matrix['Phi'] - hough_matrix['Theta']) / hough_matrix['R']).values
    
    return hough_matrix




def add_gamma(hough_matrix):
    # z_hit/(np.pi - 2*(phi_hit - theta))
    ##hough_matrix['Gamma'] = ((np.pi - 2. * (hough_matrix['Phi'] - hough_matrix['Theta'])) / hough_matrix['Z']).values
    #hough_matrix['Gamma'] = np.ones(len(hough_matrix))
    hough_matrix['Gamma'] = hough_matrix['Z']/hough_matrix['R']
    
    return hough_matrix




def digitize_column(hough_matrix, col, N, min_val=None, max_val=None):
    
    x = hough_matrix[col].values
    if min_val is not None and max_val is not None:
        bins = np.linspace(min_val, max_val, N)
    else:
        bins = np.linspace(x.min(), x.max(), N)
    bin_ids = np.digitize(x, bins)
    hough_matrix[col+'Digi'] = bin_ids
    
    return hough_matrix




def cut_min_max_bins(hough_matrix, col):
    
    digi = hough_matrix[col].values
    hough_matrix = hough_matrix[(digi > digi.min()) * (digi < digi.max())]
    
    return hough_matrix




def combine_digi(hough_matrix, columns):
    
    hough_matrix['ComboDigi'] = np.zeros(len(hough_matrix))
    
    for i_col, acol in enumerate(columns):
        digi = hough_matrix[acol]
        hough_matrix['ComboDigi'] += digi * 10**(i_col * 5)
    
    return hough_matrix




def count_combo_digi(hough_matrix):
    
    unique, indeces, counts = np.unique(hough_matrix['ComboDigi'].values, 
                                     return_counts=True, return_inverse=True)
    
    hough_matrix['ComboDigiCounts'] = counts[indeces]
    
    return hough_matrix




def reduce_matrix(hough_matrix, min_counts):
    
    return hough_matrix[hough_matrix['ComboDigiCounts'] >= min_counts].reset_index()



def sorted_groups(hough_matrix):
    
    gb = hough_matrix.groupby('ComboDigi')
    groups = np.array(list(gb.groups.values()))
    group_size = [len(i) for i in groups]
    
    groups = groups[np.argsort(group_size)[::-1]]
    
    return groups


#############################################################################################

def hough_transform_clusterer(x, y, z, N_bins_theta, N_bins_r0inv, N_bins_gamma, min_hits):
    
    hit_id = np.arange(len(x))

    r, phi, z = cartesian_to_cylindrical(x, y, z)
    hough_matrix = create_hough_matrix(hit_id, x, y, z, r, phi, N_bins_theta)
    
    hough_matrix = add_r0_inv(hough_matrix)
    hough_matrix = add_gamma(hough_matrix)
    
    hough_matrix = digitize_column(hough_matrix, 'Theta', N_bins_theta)
    hough_matrix = digitize_column(hough_matrix, 'R0Inv', N_bins_r0inv, -2./100, 2./100)
    hough_matrix = digitize_column(hough_matrix, 'Gamma', N_bins_gamma)
    
    hough_matrix = cut_min_max_bins(hough_matrix, 'R0InvDigi')
    #hough_matrix = cut_min_max_bins(hough_matrix, 'GammaDigi')
    
    hough_matrix = combine_digi(hough_matrix, ['ThetaDigi', 'R0InvDigi', 'GammaDigi'])
    #hough_matrix = combine_digi(hough_matrix, ['ThetaDigi', 'R0InvDigi'])
    hough_matrix = count_combo_digi(hough_matrix)
    hough_matrix = reduce_matrix(hough_matrix, min_hits)
    hough_matrix
    groups = sorted_groups(hough_matrix)


    track_id = 0
    track_labels = -1 * np.ones(len(hit_id))
    used = np.zeros(len(hit_id))

    matrix_hit_id = hough_matrix['HitID'].values

    for gr in groups:

        ids = matrix_hit_id[gr]
        unique_hit_ids = np.unique(ids)
        not_used_ids = unique_hit_ids[used[unique_hit_ids] == 0]

        if len(not_used_ids) >= min_hits:

            track_labels[not_used_ids] = track_id
            used[not_used_ids] += 1
            track_id +=1
            
    return hough_matrix, track_labels

In [4]:
from tracker import BaseTracker

class HoughTracker(BaseTracker):
    def __init__(self, N_bins_theta=1000, N_bins_r0inv=200, N_bins_gamma=100, min_hits=3, verbose=1):
        super().__init__()
        self.N_bins_theta = N_bins_theta
        self.N_bins_r0inv = N_bins_r0inv
        self.N_bins_gamma = N_bins_gamma
        self.min_hits = min_hits
        self.verbose = verbose
        
    def fit_generator(self, event_generator):
        pass

    def predict_generator(self, event_generator):
        sub_list = [self.predict_one_event(event_id, event) for event_id, event in event_generator]
        submission = pd.concat(sub_list)
        return submission
    
    def predict_one_event(self, event_id, event):
        if self.verbose:
            print(event_id)
        _, labels = hough_transform_clusterer(event.x.values,
                                              event.y.values,
                                              event.z.values,
                                              self.N_bins_theta,
                                              self.N_bins_r0inv,
                                              self.N_bins_gamma,
                                              self.min_hits,)
        sub = pd.DataFrame(data=np.column_stack((event.hit_id.values, labels)),
                                  columns=["hit_id", "track_id"]).astype(int)
        sub['event_id'] = event_id
        return sub
    

        

In [15]:
!ls /data/titanic_3/user/vestrade/seattle_trackml/

ls: impossible d'accéder à '/data/titanic_3/user/vestrade/seattle_trackml/': Aucun fichier ou dossier de ce type


# Tracking on the training set 

In [16]:
from dataset import load_hit_generator

data_path = "./public_data/training/"
event_generator = load_hit_generator(data_path)

In [17]:
from tracker import ParallelPredictTracker
base_tracker = HoughTracker(verbose=1)
tracker = ParallelPredictTracker(base_tracker, n_jobs=5)

** Next cell will require some time to compute !**

In [18]:
submission = tracker.predict_generator(event_generator)

50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


# Scoring

In [19]:
solution = pd.read_csv("./public_data/training/solution.csv")

In [20]:
from score import compute_score

In [21]:
%%time
score_mean, score_std, n_purity_reco, n_purity_truth = compute_score(submission, solution)

In [22]:
print('score mean:   {:f}'.format(score_mean))
print('score stddev: {:f}'.format(score_std))
print('purity_reco: {:d}'.format(n_purity_reco))
print('purity_truth: {:d}'.format(n_purity_truth))


score mean:   0.001707
score stddev: 0.015229
purity_reco: 911
purity_truth: 186


In [30]:
print(len(submission['track_id'].unique()), 'predicted particles')
print(len(solution['particle_id'].unique()), 'true particles')


2109 predicted particles
21866 true particles


In [34]:
print((submission['track_id'] == -1).sum(), 'rejected hits')
print((solution['particle_id'] == -1).sum(), 'rejected hits')


80625 rejected hits
0 rejected hits


# Create a Submission

## Tracking on validation data

In [ ]:
data_path = "./public_data/validation/"
event_generator = load_hit_generator(data_path)
submission = tracker.predict_generator(event_generator)

## Write the submission file

In [ ]:
submission.to_csv('./submission.csv')

Now just zip the file (**ONLY THE csv FILE ! NOT THE DIRECTORY**) and you can submit it to codalab.

The submission file has to be name "submission.csv"